In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

import matplotlib as mpl

/bin/bash: line 1: nvcc: command not found
torch:  2.5 ; cuda:  2.5.1
detectron2: 0.6


In [2]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("IP8M-H-NW_train", {}, "/data/mtmoore/school/CSiML_AI395T/final_project/dataset/IP8M-H-NW_coco/annotations/instances_train.json", "/data/mtmoore/school/CSiML_AI395T/final_project/dataset/IP8M-H-NW_coco/")
register_coco_instances("IP8M-H-NW_val", {}, "/data/mtmoore/school/CSiML_AI395T/final_project/dataset/IP8M-H-NW_coco/annotations/instances_train.json", "/data/mtmoore/school/CSiML_AI395T/final_project/dataset/IP8M-H-NW_coco/")

In [3]:
#t = DatasetCatalog.get("my_train")
#print(t)

In [4]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib as mpl
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = "IP8M-H-NW_train"
cfg.DATASETS.TEST = "IP8M-H-NW_val"
cfg.DATALOADER.NUM_WORKERS = 8
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 8  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.OUTPUT_DIR = "/data/mtmoore/school/CSiML_AI395T/final_project/models/detectron2/faster_rcnn_101_FPN"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[11/17 20:26:23 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[11/17 20:26:23 d2.engine.train_loop]: Starting training from iteration 0


/scratch/mtmoore/local/miniconda/envs/csiml-final-detectron2/lib/python3.12/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647378361/work/aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[11/17 20:26:43 d2.utils.events]:  eta: 0:14:22  iter: 19  total_loss: 1.01  loss_cls: 0.616  loss_box_reg: 0.1701  loss_rpn_cls: 0.1306  loss_rpn_loc: 0.03956    time: 0.8809  last_time: 0.8776  data_time: 0.0766  last_data_time: 0.0182   lr: 4.9953e-06  max_mem: 11059M
[11/17 20:27:01 d2.utils.events]:  eta: 0:14:05  iter: 39  total_loss: 0.9061  loss_cls: 0.5502  loss_box_reg: 0.1882  loss_rpn_cls: 0.1291  loss_rpn_loc: 0.03732    time: 0.8804  last_time: 0.8774  data_time: 0.0191  last_data_time: 0.0188   lr: 9.9902e-06  max_mem: 11059M
[11/17 20:27:18 d2.utils.events]:  eta: 0:13:45  iter: 59  total_loss: 0.7262  loss_cls: 0.439  loss_box_reg: 0.138  loss_rpn_cls: 0.1275  loss_rpn_loc: 0.04258    time: 0.8743  last_time: 0.8609  data_time: 0.0182  last_data_time: 0.0176   lr: 1.4985e-05  max_mem: 11059M
[11/17 20:27:36 d2.utils.events]:  eta: 0:13:24  iter: 79  total_loss: 0.5986  loss_cls: 0.3256  loss_box_reg: 0.1405  loss_rpn_cls: 0.09671  loss_rpn_loc: 0.03724    time: 0.8730 

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("IP8M-H-NW_val", ("bbox",), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "IP8M-H-NW_val")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

In [ ]:
for d in random.sample(my_dataset_train, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=balloon_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])